# **IDS using Bi-LSTM model**

## **Method used -** CNN with Bi-LSTM

### **Paper reference1 -**Efficient Deep CNN-BiLSTM Model for Network Intrusion Detection

### **Implemented By -** Sarvesh Sridhar 19BAI1057

#### **Proposed Method in Paper Reference-**
Neural Network consisting of CNN and Bi-LSTM's

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, SimpleRNN,Bidirectional, BatchNormalization,Convolution1D,MaxPooling1D, Reshape, GlobalAveragePooling1D
from tensorflow.keras.utils import to_categorical
from sklearn import metrics

In [ ]:
# Importing all datafiles
t1 = pd.read_csv('D:/college/5TH SEM/(project) AI&CS/CIC-IDS2017/main_datasets/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv')
t2 = pd.read_csv('D:/college/5TH SEM/(project) AI&CS/CIC-IDS2017/main_datasets/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv')
t3 = pd.read_csv('D:/college/5TH SEM/(project) AI&CS/CIC-IDS2017/main_datasets/Friday-WorkingHours-Morning.pcap_ISCX.csv')
# t4 = pd.read_csv('D:/college/5TH SEM/(project) AI&CS/CIC-IDS2017/main_datasets/Monday-WorkingHours.pcap_ISCX.csv')
# t5 = pd.read_csv('D:/college/5TH SEM/(project) AI&CS/CIC-IDS2017/main_datasets/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv')
# t6 = pd.read_csv('D:/college/5TH SEM/(project) AI&CS/CIC-IDS2017/main_datasets/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv')
# t7 = pd.read_csv('D:/college/5TH SEM/(project) AI&CS/CIC-IDS2017/main_datasets/Tuesday-WorkingHours.pcap_ISCX.csv')
# t8 = pd.read_csv('D:/college/5TH SEM/(project) AI&CS/CIC-IDS2017/main_datasets/Wednesday-workingHours.pcap_ISCX.csv')

df = pd.concat([t1,t2,t3])
print(df.shape)
df.head()

(703245, 79)


,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [ ]:
# Check for null values
df.isnull().values.any()

True

In [ ]:
def modify_column_names(df):
    return df.columns.str.strip().str.lower().str.replace(' ', '_')
def clean_dataset(df):
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep]

df.columns = modify_column_names(df)
df_clean = clean_dataset(df)

print("Shape before: ", df.shape)
print("Shape after:", df_clean.shape)

Shape before:  (703198, 79)
Shape after: (702718, 79)


In [ ]:
#Function to min-max normalize
def normalize(df, cols):
    result = df.copy() # do not touch the original df
    for feature_name in cols:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        if max_value > min_value:
            result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

#Dropping subclass column
attack = df_clean['label']
df_clean.drop('label', axis=1, inplace = True)

#Normalizing training set
df_final = normalize(df_clean,df_clean.columns)
df_final.head()

C:\Users\KSGS\.conda\envs\tf\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,destination_port,flow_duration,total_fwd_packets,total_backward_packets,total_length_of_fwd_packets,total_length_of_bwd_packets,fwd_packet_length_max,fwd_packet_length_min,fwd_packet_length_mean,fwd_packet_length_std,...,act_data_pkt_fwd,min_seg_size_forward,active_mean,active_std,active_max,active_min,idle_mean,idle_std,idle_max,idle_min
0,0.837225,1.333333e-07,0.000005,0.000000,0.000010,0.000000e+00,0.000242,0.002581,0.00101,0.0,...,0.000005,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.840109,1.016667e-06,0.000000,0.000004,0.000005,9.569378e-09,0.000242,0.002581,0.00101,0.0,...,0.000000,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.840124,5.416666e-07,0.000000,0.000004,0.000005,9.569378e-09,0.000242,0.002581,0.00101,0.0,...,0.000000,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.705548,3.916666e-07,0.000000,0.000004,0.000005,9.569378e-09,0.000242,0.002581,0.00101,0.0,...,0.000000,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.837194,1.333333e-07,0.000005,0.000000,0.000010,0.000000e+00,0.000242,0.002581,0.00101,0.0,...,0.000005,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
attack.unique()

array(['BENIGN', 'DDoS', 'PortScan', 'Bot'], dtype=object)

In [ ]:
#Fixing labels for training set
classlist = []

benign = ("BENIGN")
dos = ("DoS Hulk","DDoS", "DoS slowloris", "DoS Slowhttptest", "DoS Hulk", "DoS GoldenEye", "Heartbleed")
portscan = ("PortScan")
bot = ("Bot")
infiltration = ("Infiltration")
webattack = ("Web Attack � Brute Force", "Web Attack � XSS", "Web Attack � Sql Injection")
brute_force = ("FTP-Patator", "SSH-Patator")

for item in attack:
    if item in benign:
        classlist.append('0')
    elif item in dos:
        classlist.append("1")
    elif item in portscan:
        classlist.append("2")
    elif item in bot:
        classlist.append("3")
    elif item in infiltration:
        classlist.append("4")
    elif item in webattack:
        classlist.append("5")
    elif item in brute_force:
        classlist.append("6")
    else:
        print(item, "not found -_-")
        
#Appending class column to training set
df_final["class"] = classlist
df_final.head()

,destination_port,flow_duration,total_fwd_packets,total_backward_packets,total_length_of_fwd_packets,total_length_of_bwd_packets,fwd_packet_length_max,fwd_packet_length_min,fwd_packet_length_mean,fwd_packet_length_std,...,min_seg_size_forward,active_mean,active_std,active_max,active_min,idle_mean,idle_std,idle_max,idle_min,class
0,0.837225,1.333333e-07,0.000005,0.000000,0.000010,0.000000e+00,0.000242,0.002581,0.00101,0.0,...,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.840109,1.016667e-06,0.000000,0.000004,0.000005,9.569378e-09,0.000242,0.002581,0.00101,0.0,...,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.840124,5.416666e-07,0.000000,0.000004,0.000005,9.569378e-09,0.000242,0.002581,0.00101,0.0,...,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.705548,3.916666e-07,0.000000,0.000004,0.000005,9.569378e-09,0.000242,0.002581,0.00101,0.0,...,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.837194,1.333333e-07,0.000005,0.000000,0.000010,0.000000e+00,0.000242,0.002581,0.00101,0.0,...,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [ ]:
df_final['class'].value_counts()

0    413933
2    158804
1    128025
3      1956
Name: class, dtype: int64

In [ ]:
x = df_final.drop('class', axis = 1)
y = df_final["class"]

print("Shape of x: ", x.shape)
print("Shape of y: ", y.shape)

Shape of x:  (702718, 78)
Shape of y:  (702718,)


In [ ]:
oos_pred = []

kfold = StratifiedKFold(n_splits=6,shuffle=True,random_state=42)
kfold.get_n_splits(x,y)

6

In [ ]:
import tensorflow as tf
from keras.models import Sequential

model = Sequential()
model.add(Convolution1D(64, kernel_size = 64, padding = 'same', activation = 'relu', input_shape = (78,1)))
model.add(MaxPooling1D(pool_size  =(10)))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(64,return_sequences = False)))
model.add(Reshape((128,1), input_shape = (128,)))
model.add(MaxPooling1D(pool_size = (5)))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(128,return_sequences = False)))
model.add(Dropout(0.5))
model.add(Dense(4))
model.add(Activation('softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
for layer in model.layers:
    print(layer.output_shape)

(None, 78, 64)
(None, 7, 64)
(None, 7, 64)
(None, 128)
(None, 128, 1)
(None, 25, 1)
(None, 25, 1)
(None, 256)
(None, 256)
(None, 4)
(None, 4)


In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 78, 64)            4160      
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 7, 64)            0         
 1D)                                                             
                                                                 
 batch_normalization_2 (Batc  (None, 7, 64)            256       
 hNormalization)                                                 
                                                                 
 bidirectional_2 (Bidirectio  (None, 128)              66048     
 nal)                                                            
                                                                 
 reshape_1 (Reshape)         (None, 128, 1)            0         
                                                      

In [ ]:
for train_index, test_index in kfold.split(x, y):
    # split
    train_X, test_X = x.iloc[train_index], x.iloc[test_index]
    train_y, test_y = y.iloc[train_index], y.iloc[test_index]
    
    x_columns_train = df_final.columns.drop('class')
    x_train_array = train_X[x_columns_train].values
    x_train_1=np.reshape(x_train_array, (x_train_array.shape[0], x_train_array.shape[1], 1))
    
    dummies = pd.get_dummies(train_y)
    outcomes = dummies.columns
    num_classes = len(outcomes)
    y_train_1 = dummies.values
    
    x_columns_test = df_final.columns.drop('class')
    x_test_array = test_X[x_columns_test].values
    x_test_2=np.reshape(x_test_array, (x_test_array.shape[0], x_test_array.shape[1], 1))
    
    dummies_test = pd.get_dummies(test_y) # Classification
    outcomes_test = dummies_test.columns
    num_classes = len(outcomes_test)
    y_test_2 = dummies_test.values
    
    with tf.device('/GPU:0'):
        history = model.fit(x_train_1, y_train_1,validation_data=(x_test_2,y_test_2), epochs=10, batch_size = 200)
    
    pred = model.predict(x_test_2)
    pred = np.argmax(pred,axis=1)
    y_eval = np.argmax(y_test_2,axis=1)
    score = metrics.accuracy_score(y_eval, pred)
    oos_pred.append(score)
    print("Validation score: {}".format(score))

Epoch 1/10
2928/2928 [==============================] - 99s 23ms/step - loss: 0.0372 - accuracy: 0.9889 - val_loss: 0.0208 - val_accuracy: 0.9939
Epoch 2/10
2928/2928 [==============================] - 65s 22ms/step - loss: 0.0261 - accuracy: 0.9926 - val_loss: 0.0191 - val_accuracy: 0.9940
Epoch 3/10
2928/2928 [==============================] - 62s 21ms/step - loss: 0.0158 - accuracy: 0.9950 - val_loss: 0.0126 - val_accuracy: 0.9960
Epoch 4/10
2928/2928 [==============================] - 60s 20ms/step - loss: 0.0127 - accuracy: 0.9957 - val_loss: 0.0103 - val_accuracy: 0.9962
Epoch 5/10
2928/2928 [==============================] - 65s 22ms/step - loss: 0.0115 - accuracy: 0.9962 - val_loss: 0.0103 - val_accuracy: 0.9954
Epoch 6/10
2928/2928 [==============================] - 64s 22ms/step - loss: 0.0105 - accuracy: 0.9966 - val_loss: 0.0123 - val_accuracy: 0.9954
Epoch 7/10
2928/2928 [==============================] - 64s 22ms/step - loss: 0.0096 - accuracy: 0.9969 - val_loss: 0.0106 -

In [ ]:
oos_pred

[0.9977800546448088,
 0.9971226092896175,
 0.9966188524590164,
 0.9971140710382513,
 0.9979764171483705,
 0.99814718363374]

In [ ]:
cm = confusion_matrix(y_eval, pred, labels=[0,1,2,3,4])

def count_unique(keys):
    uniq_keys = np.unique(keys)
    bins = uniq_keys.searchsorted(keys)
    return [uniq_keys, np.bincount(bins)]

y_pred = np.array(pred)
labels = print(dummies_test.columns)
print(count_unique(y_pred)[0])
count_unique(y_pred)[1]

print(cm)

Index(['0', '1', '2', '3'], dtype='object')
[0 1 2 3]
[[68978     6     0     4     0]
 [    3 21335     0     0     0]
 [   15     0 26452     0     0]
 [  189     0     0   137     0]
 [    0     0     0     0     0]]


In [ ]:
from sklearn.metrics import classification_report
print('\nClassification Report\n')
print(classification_report(y_eval, pred, target_names = labels))


Classification Report

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     68988
           1       1.00      1.00      1.00     21338
           2       1.00      1.00      1.00     26467
           3       0.97      0.42      0.59       326

    accuracy                           1.00    117119
   macro avg       0.99      0.85      0.90    117119
weighted avg       1.00      1.00      1.00    117119



In [ ]:
dummies = pd.get_dummies(y)
outcomes = dummies.columns
num_classes = len(outcomes)
y_final = dummies.values
    
y_pred = model.predict(x.values)
pred = np.argmax(y_pred ,axis=1)
y_eval = np.argmax(y_final,axis=1)
score = metrics.accuracy_score(y_eval, pred)

print("Accuracy :", score)

Accuracy : 0.9981016567100885
